In [39]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

In [40]:
# import data

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_data.pkl', 'rb') as f:
    books_data = pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/ratings_data.pkl', 'rb') as f:
    ratings_data= pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/users_data.pkl', 'rb') as f:
    users_data= pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_ratings.pkl', 'rb') as f:
    books_ratings= pickle.load(f)

    users_data = pd.DataFrame(users_data)

In [41]:
# will take only first 1000, otherwise the data are too big
users_data = users_data.head(1000)
users_data

,User-ID,Location,Age
0,1,"nyc, new york, usa",0
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",0
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",0
...,...,...,...
995,996,"bellefonte, pennsylvania, usa",0
996,997,"meeteetse, wyoming, usa",0
997,998,"champs sur marne, île de france, france",0
998,999,"newark, delaware, usa",21


In [42]:
# use TdifVectorizer to transform Location description to number
location_tfidf = TfidfVectorizer().fit_transform(users_data['Location'])

# create a DataFrame with the cosine similarities of the Locations of the users
cosine_sim = pd.DataFrame(cosine_similarity(location_tfidf))

cosine_sim


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.038257,0.0,0.0,0.0,0.029843,0.039204,0.0,0.032330,0.0,...,0.030868,0.0,0.0,0.034895,0.121419,0.034338,0.029849,0.0,0.030724,0.0
1,0.038257,1.000000,0.0,0.0,0.0,0.193259,0.047147,0.0,0.038880,0.0,...,0.037122,0.0,0.0,0.041964,0.000000,0.041295,0.035896,0.0,0.036949,0.0
2,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
3,0.000000,0.000000,0.0,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.034338,0.041295,0.0,0.0,0.0,0.032212,0.042317,0.0,0.034897,0.0,...,0.033319,0.0,0.0,0.037665,0.000000,1.000000,0.032219,0.0,0.033163,0.0
996,0.029849,0.035896,0.0,0.0,0.0,0.028001,0.036785,0.0,0.030335,0.0,...,0.028963,0.0,0.0,0.032741,0.000000,0.032219,1.000000,0.0,0.028828,0.0
997,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0
998,0.030724,0.036949,0.0,0.0,0.0,0.028822,0.037863,0.0,0.031224,0.0,...,0.029812,0.0,0.0,0.033701,0.000000,0.033163,0.028828,0.0,1.000000,0.0


In [43]:
def user_recommendation(UserID):
    # get the age of the user
    user_age = users_data.loc[users_data['User-ID'] == UserID]['Age']

    # get the Location_Cosine from the cosine_sim DataFrame
    users_locations = pd.DataFrame(cosine_sim.loc[UserID - 1])

    # concat the users with the Location_Cosine data
    user_df = pd.concat([users_data[['Age']], users_locations], axis=1)

    # rename the column as Location_Cosine
    user_df.rename(columns={user_df.columns[1]: 'Location_Cosine'}, inplace=True)

    # get the first 5 best cosine results
    top_5_indices = user_df.sort_values(by = ['Location_Cosine'], ascending = [False]).head(5).index.tolist()

    # get the User-ID
    users = users_data.loc[top_5_indices]['User-ID']

    # get the ISBN from the recommended users with book with higher rating than 5
    ISBN = ratings_data.loc[(ratings_data['User-ID'].isin(users)) & (ratings_data['Book-Rating'] > 5)]['ISBN']

    # get the books data

    books = books_data.loc[books_data['ISBN'].isin(ISBN)]

    return books


In [44]:
''' test not working
UserID = 254

user_age = users_data.loc[users_data['User-ID'] == UserID]['Age']

users_locations = pd.DataFrame(cosine_sim.loc[UserID - 1])

user_df = pd.concat([users_data[['Age']], users_locations], axis=1)

user_df.rename(columns={user_df.columns[1]: 'Location_Cosine'}, inplace=True)

user_df['Age'] = user_df['Age'].astype(int)

user_df['Age'] = user_df['Age'] - user_age

user_df
'''

" test not working\nUserID = 254\n\nuser_age = users_data.loc[users_data['User-ID'] == UserID]['Age']\n\nusers_locations = pd.DataFrame(cosine_sim.loc[UserID - 1])\n\nuser_df = pd.concat([users_data[['Age']], users_locations], axis=1)\n\nuser_df.rename(columns={user_df.columns[1]: 'Location_Cosine'}, inplace=True)\n\nuser_df['Age'] = user_df['Age'].astype(int)\n\nuser_df['Age'] = user_df['Age'] - user_age\n\nuser_df\n"

In [45]:
ratings_data.loc[ratings_data['ISBN'] == '0590353403']

,User-ID,ISBN,Book-Rating
10279,254,0590353403,9
14193,2033,0590353403,9
14755,2179,0590353403,10
22576,4809,0590353403,8
27162,6431,0590353403,10
...,...,...,...
1139566,274061,0590353403,0
1145158,275401,0590353403,9
1146002,275843,0590353403,5
1147867,276024,0590353403,9


In [46]:
users_data.loc[253]

User-ID                             254
Location    minneapolis, minnesota, usa
Age                                  24
Name: 253, dtype: object

In [47]:
user_recommendation(254)

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
269,0373836023,GEORGETT.HEYER,POWDER AND PATCH,2004,HARLEQUIN,EN
274,037582345X,PHILI.PULLMAN,"THE GOLDEN COMPASS HIS DARK MATERIALS, BOOK 1",2002,KNOPF BOOKS FOR YOUNG READERS,EN
275,0375823468,PHILI.PULLMAN,"THE SUBTLE KNIFE HIS DARK MATERIALS, BOOK 2",2002,KNOPF BOOKS FOR YOUNG READERS,EN
276,0380730448,SU.TOWNSEND,THE ADRIAN MOLE DIARIES THE SECRET DIARY OF A...,1997,PERENNIAL,FR
277,0380973839,RA.BRADBURY,MARTIAN CHRONICLES,1997,WILLIAM MORROW,EN
282,0451409256,ISABE.WOLFF,MAKING MINTY MALONE,2000,ONYX BOOKS,EN
285,0451526341,GEORG.ORWELL,ANIMAL FARM,2004,SIGNET,EN
300,156971620X,NEI.GAIMAN,HARLEQUIN VALENTINE,2002,DARK HORSE COMICS,EN
356,0142001740,SU.MON.KIDD,THE SECRET LIFE OF BEES,2003,PENGUIN BOOKS,FR
485,0671021001,WALL.LAMB,SHE'S COME UNDONE OPRAH'S BOOK CLUB,1998,POCKET,EN
